In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
class PINN(nn.Module):
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation(),
                       ])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation(),
                            ]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)

    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

In [3]:
def plot_nn_results(input_data, output_data, predicted_data, loss=None):
    """
    Fonction pour tracer les résultats d'un réseau neuronal à une seule entrée et une seule sortie.

    Arguments :
    - input_data : Les données d'entrée
    - output_data : Les données de sortie réelles
    - predicted_data : Les données de sortie prédites par le réseau neuronal
    """

    # Création d'une figure et un axe
    fig, ax = plt.subplots()

    # Tracer les données réelles
    ax.plot(input_data, output_data, 'b', label='Réel')

    # Tracer les données prédites
    ax.plot(input_data, predicted_data, 'r', label='Prédiction')

    if loss:
      ax.set_title(f'Loss = {loss}')

    # Ajout d'une légende
    ax.legend()

    # Ajout des titres aux axes
    ax.set_xlabel('x')
    ax.set_ylabel('u')

    # Affichage du graphique
    plt.show()

In [4]:
def solution_exact(E, I, L, p, x):
    """
    Fonction pour calculer une solution exacte à l'équation donnée.

    Arguments :
    - E : Valeur du module de young E
    - A : Valeur de la section A
    - I : Moment quadratique
    - p : Valeur de la force
    - x : Valeur des entrées x

    Retour :
    - y : Valeur de la solution exacte de l'équation
    """

    # Calculer la valeur de la solution exacte
    y = p*(-x**4 + 2.5*L*(x**3) - (3/2)*L**2*x**2) / (24 * E * I)

    # Retourner la valeur de la solution
    return y


In [ ]:
L = 2.7 # longueur de la barre
x_data = torch.linspace(0, L, 500).view(-1,1)
y_data = solution_exact(200e9,0.000038929334,L,60e3, x_data)
x = x_data.detach()
y = y_data.detach()
plt.plot(x, y, label="Solution exacte")
plt.legend()
plt.show()

In [ ]:
import time
# Définir les valeurs des coefficients
E, I, L, p = 200e9, 0.000038929334, 2.7, 60e3

# Définir les emplacements d'échantillonnage sur le domaine du problème
x_physics = torch.linspace(0, L, 500).view(-1, 1).requires_grad_(True)
x_0 = torch.tensor(0.).view(-1,1).requires_grad_(True)
x_L = torch.tensor(L).view(-1,1).requires_grad_(True)

# Fixer la graine aléatoire pour la reproductibilité
torch.manual_seed(123)

# Créer une instance du modèle PINN avec les paramètres donnés
model = PINN(1, 1, 20, 1)

# Définir l'optimiseur avec Adam et le taux d'apprentissage de 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Enregistrement du temps de départ
temps_debut = time.time()
# Boucle d'entraînement
for i in range(15000):
    # Réinitialiser les gradients à zéro
    optimizer.zero_grad()

    yh = model(x_data)

    # Calculer la "perte de physique"
    yhp = model(x_physics)
    dudx = torch.autograd.grad(yhp, x_physics, torch.ones_like(yhp), create_graph=True)[0]  # calculer dy/dx
    d2udx2 = torch.autograd.grad(dudx, x_physics, torch.ones_like(dudx), create_graph=True)[0]  # calculer d2y/dx2
    d3udx3 = torch.autograd.grad(d2udx2, x_physics, torch.ones_like(d2udx2), create_graph=True)[0]  # calculer d3y/dx3
    d4udx4 = torch.autograd.grad(d3udx3, x_physics, torch.ones_like(d3udx3), create_graph=True)[0]  # calculer d4y/dx4


    residus = d4udx4 + p/(E*I)   # calculer le résidu de l'équation différentielle
    loss2 = torch.mean(residus ** 2)

    # L'erreur au limites
    u_0 = model(x_0)
    du0dx = torch.autograd.grad(u_0, x_0, torch.ones_like(u_0), create_graph=True)[0]
    d2u0dx2 = torch.autograd.grad(du0dx, x_0, torch.ones_like(du0dx), create_graph=True)[0]


    loss3 = torch.squeeze(u_0**2) + torch.squeeze(du0dx**2)

    u_L = model(x_L)
    duLdx = torch.autograd.grad(u_L, x_L, torch.ones_like(u_L), create_graph=True)[0]
    d2uLdx2 = torch.autograd.grad(duLdx, x_L, torch.ones_like(duLdx), create_graph=True)[0]
    d3uLdx3 = torch.autograd.grad(d2uLdx2, x_L, torch.ones_like(d2uLdx2), create_graph=True)[0]
    loss4 = torch.squeeze(d2uLdx2**2) + torch.squeeze(u_L**2)


    # Rétropropagation pour calculer les gradients
    loss = loss2 + loss3 + loss4 # ajouter les trois termes de perte ensemble
    
    loss.backward()

    # Mettre à jour les poids du modèle en utilisant l'optimiseur
    optimizer.step()

    # Afficher le graphique des résultats pendant l'entraînement
    if (i + 1) % 150 == 0:
        # Détacher les prédictions yh du graphe de calcul
        yh = model(x).detach()
        xp = x_physics.detach()

        # Tracer les résultats du modèle
        plot_nn_results(x_data, y_data, yh, loss)
# Enregistrement du temps de fin
temps_fin = time.time()

# Calculez le temps écoulé
temps_ecoule = temps_fin - temps_debut


In [ ]:
params = model.named_parameters()
for param_name,param in params:
    print(f"name: {param_name}\nvalue: {param}")